## Make a simple synthetic dataset to test hierarchical merge in FOF algorithm

#### the idea is this: 

* after the local FOF stage, each partition reports the particles it holds in the overlap region
* do a reduceByKey or treeAggregate of some sort to collect the groups belonging to the same particles
* produce a mapping of $G -> G_1$ and distribute to all hosts in form of broadcast lookup table

In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

import sys
sys.setrecursionlimit(sys.getrecursionlimit()*10)
import matplotlib.patches as patches

plt.style.use('fivethirtyeight')

In [2]:
import os
os.environ['SPARK_HOME'] = os.path.join(os.path.expanduser('~'), 'spark')
import findspark
findspark.init()

In [3]:
def plot_rectangle(rec, ax=None):
    if ax is None: 
        ax = plt.subplot(aspect='equal')
    
    if isinstance(rec, (list, tuple)):
        for r in rec: 
            plot_rectangle(r,ax)
    
    else:
        size = (rec.maxes-rec.mins)
        ax.add_patch(patches.Rectangle(rec.mins, size[0], size[1], fill=False, zorder=-1))
    plt.draw()
    plt.show()

## Start Spark

In [4]:
import os
os.environ['SPARK_CONF_DIR'] = './conf'
os.environ['SPARK_DRIVER_MEMORY'] = '8G'

In [5]:
import pyspark
from pyspark import SparkContext, SparkConf
import pynbody
import sparkhpc

In [152]:
%%writefile job.template
#!/bin/sh
#BSUB -J {jobname}
#BSUB -W {walltime} # runtime to request
#BSUB -o {jobname}-%J.log # output extra o means overwrite
#BSUB -n {ncores} # requesting ncores cores
#BSUB -R "rusage[mem={memory}, scratch=10000] span[hosts=-1]" # take any available core with mem MB of memory

# setup the spark paths
export SPARK_HOME={spark_home}
export SPARK_LOCAL_DIRS=$__LSF_JOB_TMPDIR__
export LOCAL_DIRS=$SPARK_LOCAL_DIRS
export SPARK_WORKER_DIR=$__LSF_JOB_TMPDIR__/work
#export SPARK_CONF_DIR=/cluster/home/roskarr/Projects/spark-fof/conf

sparkcluster launch --memory {memory}M


Overwriting job.template


In [153]:
sc.stop()
sj.stop()

INFO:sparkhpc:Job <33425570> is being terminated



In [154]:
#sj = sparkhpc.sparkjob.LSFSparkJob(ncores=27, memory=5500, walltime='4:00', template='./job.template')
sj = sparkhpc.sparkjob.LSFSparkJob(ncores=49,memory=12000,walltime='1:00', template='./job.template')
#sj = sparkhpc.sparkjob.LSFSparkJob(clusterid=0)

In [155]:
sj.wait_to_start()

INFO:sparkhpc:Submitted cluster 0


In [159]:
sc.stop()

In [160]:
sc = sparkhpc.start_spark(master=sj.master_url, spark_conf='../conf', profiling=True, executor_memory='11500M')

In [163]:
sc.defaultParallelism

49

In [162]:
sj

Job ID,Number of cores,Status,Spark UI,Spark URL,33439584,49,running,http://10.205.7.57:8080,spark://e1121.hpc-lca.ethz.ch:7077


### Run FOF 

In [164]:
import spark_fof

In [167]:
reload(spark_fof.spark_fof)
reload(spark_fof)
reload(spark_fof.spark_fof_c)

<module 'spark_fof.spark_fof_c' from '/cluster/project/sis/ri/roskarr/spark-fof/spark_fof/spark_fof_c.so'>

In [ ]:
sc.stop()
sj.stop()

In [168]:
%%time
#path = '/cluster/home/roskarr/work/2Tlc-final/'
path = '/cluster/home/roskarr/projects/euclid/2Tlc-final/'

# domain parameters
diff = np.float32(0.033068776)
global_min = -31*diff
global_max = 31*diff

dom_maxs = np.array([global_max]*3, dtype=np.float64)
dom_mins = np.array([global_min]*3, dtype=np.float64)

#tau = diff*5./125.
tau = 0.2/12600
buffer_tau = diff*5./150.

fof_analyzer = spark_fof.spark_fof.LCFOFAnalyzer(sc, path, 64, 62, tau, dom_mins, dom_maxs, blockids=[30,31,32,33,34,35,36,37], buffer_tau=tau*2)

Number of input files:  343
Total number of particles:  <built-in method values of dict object at 0x2b237419f5c8>
CPU times: user 7.92 s, sys: 315 ms, total: 8.24 s
Wall time: 8.27 s


In [169]:
%%time
len(fof_analyzer.groups)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job 0 cancelled because Stage 2 was cancelled
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1454)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1393)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply$mcVI$sp(DAGScheduler.scala:1381)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1380)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1380)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:234)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:1380)
	at org

In [68]:
%%time
from itertools import izip 

nMinMembers = 64

def count_groups_partition(particle_arrays, gr_map_inv_b, nMinMembers): 
    p_arr = np.concatenate(list(particle_arrays))
    gs, counts = np.unique(p_arr['iGroup'], return_counts=True)
    gr_map_inv = gr_map_inv_b.value
    return ((g,cnt) for g,cnt in izip(gs,counts) if (g in gr_map_inv) or (cnt >= nMinMembers))

merged_rdd = fof_analyzer.merged_rdd

group_merge_map = fof_analyzer.group_merge_map
gr_map_inv = {v:k for (k,v) in group_merge_map.iteritems()}
gr_map_inv_b = sc.broadcast(gr_map_inv)

# first, get rid of ghost particles
no_ghosts_rdd = merged_rdd.map(lambda p: p[np.where(p['is_ghost'] != spark_fof.spark_fof.GHOST_PARTICLE_COPY)[0]])

# count up the number of particles in each group in each partition
group_counts = no_ghosts_rdd.mapPartitions(lambda p_arrs: count_groups_partition(p_arrs, gr_map_inv_b, nMinMembers)).cache()
group_counts.count()

CPU times: user 3.55 s, sys: 206 ms, total: 3.76 s
Wall time: 9min 9s


In [69]:
# merge the groups that reside in multiple domains
merge_group_counts = (group_counts.filter(lambda (g,cnt): g in gr_map_inv_b.value)
                                  .reduceByKey(lambda a,b: a+b)
                                  .filter(lambda (g,cnt): cnt>=nMinMembers))

In [70]:
# combine the group counts
total_group_counts = (group_counts.filter(lambda (gid,cnt): gid not in gr_map_inv_b.value) + merge_group_counts).collect()

In [71]:
fof_analyzer.fof_rdd

PythonRDD[460] at RDD at PythonRDD.scala:48

In [72]:
sqc = sparkhpc.get_sqc(sc)

In [73]:
from pyspark.sql import Row

In [74]:
spark_fof.spark_fof.pdt

dtype({'names':['pos','is_ghost','iOrder','iGroup'], 'formats':[('<f4', (3,)),'<i4','<i4','<i8'], 'offsets':[0,12,16,24], 'itemsize':32}, align=True)

In [82]:
def pid_gid_ghost(p):
    return Row(gid=int(p['iGroup']),pid=int(p['iOrder']),is_ghost=int(p['is_ghost']))

In [83]:
fof_gpghost = sqc.createDataFrame(fof_analyzer.fof_rdd.flatMap(lambda p: p).map(pid_gid_ghost))

In [86]:
fof_rdd = fof_analyzer.fof_rdd

In [93]:
f = fof_rdd.first()

In [114]:
fof_gpghost = sqc.createDataFrame(fof_rdd.flatMap(lambda arr: list(arr[['iGroup','iOrder','is_ghost']]))
                                               .map(pid_gid_ghost))

In [115]:
fof_gpghost.show()

+---+--------+---+
|gid|is_ghost|pid|
+---+--------+---+
| 12|       1|  0|
| 12|       1|  1|
| 12|       1|  2|
| 12|       1|  3|
| 12|       1|  4|
| 12|       1|  5|
| 25|       1|  6|
| 25|       1|  7|
| 25|       1|  8|
| 25|       0|  9|
| 25|       0| 10|
| 25|       0| 11|
| 25|       0| 12|
| 25|       0| 13|
| 25|       0| 14|
| 25|       0| 15|
| 25|       0| 16|
| 25|       0| 17|
|241|       0| 18|
|241|       0| 19|
+---+--------+---+
only showing top 20 rows



In [121]:
groups = fof_gpghost.groupBy('pid')

In [122]:
linked_groups = groups.count().filter('count > 1')

In [123]:
%%time
linked_groups.count()

Py4JJavaError: An error occurred while calling o1981.count.
: org.apache.spark.SparkException: Job 70 cancelled because Stage 728 was cancelled
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1454)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1393)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply$mcVI$sp(DAGScheduler.scala:1381)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1380)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1380)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:234)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:1380)
	at org.apache.spark.scheduler.DAGSched

In [95]:
7550*.02/27

5.592592592592593

In [84]:
fof_gpghost.count()

Py4JJavaError: An error occurred while calling o1493.count.
: org.apache.spark.SparkException: Job 40 cancelled because Stage 636 was cancelled
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1454)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1393)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply$mcVI$sp(DAGScheduler.scala:1381)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1380)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1380)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:234)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:1380)
	at org.apache.spark.scheduler.DAGSched

In [24]:
def ghost_counts(index, iterator): 
    nghosts = 0
    nghosts_copy = 0
    nother = 0
    for arr in iterator: 
        nghosts += len(np.where(arr['is_ghost']==1)[0])
        nghosts_copy += len(np.where(arr['is_ghost']==2)[0])
        nother += len(np.where(arr['is_ghost']==0)[0])
    yield index,nother,nghosts,nghosts_copy

In [25]:
fof_rdd = fof_analyzer.fof_rdd

In [26]:
fof_rdd.cache().count()

15456

In [27]:
%load_ext memory_profiler

In [29]:
%memit groups = fof_analyzer._get_level_map()

peak memory: 5978.56 MiB, increment: 5271.09 MiB


In [84]:
import cPickle

In [86]:
cPickle.dump(groups, open('groups_map.pickle','w'))

In [ ]:
%%time
final_rdd = fof_analyzer.final_fof_rdd
final_rdd.count()

In [87]:
len(groups)

2876775

In [87]:
len(fof_analyzer2.groups)

1267851

In [89]:
sc.stop()
sj.stop()

INFO:sparkhpc:Job <33251605> is being terminated



In [88]:
sc.show_profiles()

Profile of RDD<id=171>
         1593 function calls in 0.014 seconds

   Ordered by: internal time, cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       81    0.002    0.000    0.011    0.000 rdd.py:1713(add_shuffle_key)
       27    0.002    0.000    0.005    0.000 broadcast.py:82(load)
       27    0.002    0.000    0.002    0.000 {cPickle.load}
       27    0.001    0.000    0.001    0.000 {open}
       27    0.001    0.000    0.001    0.000 {cPickle.dumps}
       54    0.001    0.000    0.001    0.000 serializers.py:143(_write_with_length)
       27    0.001    0.000    0.001    0.000 serializers.py:217(load_stream)
       27    0.000    0.000    0.001    0.000 rdd.py:61(portable_hash)
       27    0.000    0.000    0.006    0.000 broadcast.py:92(value)
       27    0.000    0.000    0.014    0.001 worker.py:165(process)
       54    0.000    0.000    0.001    0.000 serializers.py:155(_read_with_length)
       27    0.000    0.000    0.013